<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/get_corr_map_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我打算先建立一个 openAI(预先存储), 和bge-medium和bge-small之间相互转化的代码

- [x] 从google drive进行数据读取
- [x] 进行batch循环，调通基础的batch embedding抽取
- [x] 判断哪些模型之间的数据是需要被记录的，记录x'x和x'y
- [ ] 计算伪逆并进行初步的存储


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/unpublic

ChatHaruhi_Waifu_extended.jsonl  pseudo_inverses.pkl  zhwiki_2k_embedding.jsonl
erotics.jsonl			 wiki_embedding.zip   星绘.jsonl


为我实现一段python代码。把/content/drive/MyDrive/unpublic/wiki_embedding.zip 拷贝到/content下

并且解压缩这个zip文件到/content/input

In [ ]:
import shutil
import zipfile
import os

def copy_and_unzip(source_path, dest_path, unzip_dir):
    # 检查目标路径是否存在，如果不存在则创建
    if not os.path.exists(unzip_dir):
        os.makedirs(unzip_dir)

    # 拷贝文件
    shutil.copy(source_path, dest_path)

    # 解压缩文件
    with zipfile.ZipFile(dest_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

# 定义源文件路径和目标路径
source_file = '/content/drive/MyDrive/unpublic/wiki_embedding.zip'
destination_file = '/content/wiki_embedding.zip'
unzip_directory = '/content/input'

# 调用函数
copy_and_unzip(source_file, destination_file, unzip_directory)


为我实现一段python代码，依次读取/content/input/wiki下的所有json文件

保存下来之后保存到datas这个list中

In [ ]:
import os
import json
import time

# 指定目录
directory = '/content/input/wiki'

openai_embeddings = []
texts = []

count = 0
batch_size = 2000  # 每2000个样本打印一次信息
start_time = time.time()  # 开始计时

# 遍历指定目录
for filename in os.listdir(directory):
    # 检查文件是否为JSON文件
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)
        # 打开并读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            if "input" in data and "embedding" in data:
                embedding = data["embedding"]
                if len(embedding) != 1536:
                    continue
                input = data["input"]
                openai_embeddings.append(embedding)
                texts.append(input)
                count += 1

                # 每增加batch_size个样本，打印一次信息
                if count % batch_size == 0:
                    elapsed_time = time.time() - start_time  # 计算已过时间
                    samples_per_minute = count / (elapsed_time / 60)  # 计算每分钟样本数
                    print(f"已处理 {count} 个样本, 每分钟大约处理 {samples_per_minute:.2f} 个样本。")

                if count > 100000:
                    break

# 可以在循环结束后打印总体信息
total_time = time.time() - start_time
total_samples_per_minute = count / (total_time / 60)
print(f"总共处理了 {count} 个样本, 总共耗时 {total_time:.2f} 秒, 平均每分钟处理 {total_samples_per_minute:.2f} 个样本。")

已处理 2000 个样本, 每分钟大约处理 158716.28 个样本。
已处理 4000 个样本, 每分钟大约处理 166236.60 个样本。
已处理 6000 个样本, 每分钟大约处理 171385.87 个样本。
已处理 8000 个样本, 每分钟大约处理 172482.87 个样本。
已处理 10000 个样本, 每分钟大约处理 175296.54 个样本。
已处理 12000 个样本, 每分钟大约处理 175376.75 个样本。
已处理 14000 个样本, 每分钟大约处理 171981.59 个样本。
已处理 16000 个样本, 每分钟大约处理 172653.37 个样本。
已处理 18000 个样本, 每分钟大约处理 173105.25 个样本。
已处理 20000 个样本, 每分钟大约处理 172878.17 个样本。
已处理 22000 个样本, 每分钟大约处理 170700.89 个样本。
已处理 24000 个样本, 每分钟大约处理 171463.02 个样本。
已处理 26000 个样本, 每分钟大约处理 172606.14 个样本。
已处理 28000 个样本, 每分钟大约处理 171785.67 个样本。
已处理 30000 个样本, 每分钟大约处理 171036.83 个样本。
已处理 32000 个样本, 每分钟大约处理 171066.43 个样本。
已处理 34000 个样本, 每分钟大约处理 171379.23 个样本。
已处理 36000 个样本, 每分钟大约处理 171999.60 个样本。
已处理 38000 个样本, 每分钟大约处理 164902.33 个样本。
已处理 40000 个样本, 每分钟大约处理 165941.45 个样本。
已处理 42000 个样本, 每分钟大约处理 166817.86 个样本。
已处理 44000 个样本, 每分钟大约处理 167430.92 个样本。
已处理 46000 个样本, 每分钟大约处理 167885.26 个样本。
已处理 48000 个样本, 每分钟大约处理 166981.01 个样本。
已处理 50000 个样本, 每分钟大约处理 167587.49 个样本。
已处理 52000 个样本, 每分钟大约处理 168070.60 个样本。
已处理 54000 个样本, 每


我希望为这段代码增加一个计时功能，count每增加2000个打印一次，每次打印count的数量以及估计每分钟可以新增的样本个数

In [ ]:
%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

from EmbedAdapter.utils import get_general_embeddings

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 160 (delta 21), reused 23 (delta 10), pack-reused 115
Receiving objects: 100% (160/160), 36.82 MiB | 32.08 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/Embed-Adapter


把bge家族的代码放过来

TODO: 把这个代码移动到repo里

对embedding函数的测试

测试这两个

BAAI/bge-small-en-v1.5

BAAI/bge-base-en-v1.5

In [ ]:
test_batch_n = 4

test_batch = texts[0:test_batch_n]

bge_large_15 = get_general_embeddings(test_batch, "BAAI/bge-large-en-v1.5", return_tensor = True)

bge_small_15 = get_general_embeddings(test_batch, "BAAI/bge-small-en-v1.5", return_tensor = True)

bge_base_15 = get_general_embeddings(test_batch, "BAAI/bge-base-en-v1.5", return_tensor = True)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

我们现在开始来正规写程序

In [ ]:
openai_config = {
    "name":"openai",
    "long_name":"openai",
    "embeddings":openai_embeddings, # 预先抽取的
    "batch_embed_fun":None
}

def bge_large_en_fun( texts ):
    return get_general_embeddings(texts, "BAAI/bge-large-en-v1.5", return_tensor = True)

bge_large_config = {
    "name":"bge_large_en_15",
    "long_name":"BAAI/bge-large-en-v1.5",
    "embeddings":None,
    "batch_embed_fun":bge_large_en_fun
}


def bge_small_en_fun( texts ):
    return get_general_embeddings(texts, "BAAI/bge-small-en-v1.5", return_tensor = True)

bge_small_config = {
    "name":"bge_small_en_15",
    "long_name":"BAAI/bge-small-en-v1.5",
    "embeddings":None,
    "batch_embed_fun":bge_small_en_fun
}

def bge_base_en_fun( texts ):
    return get_general_embeddings(texts, "BAAI/bge-base-en-v1.5", return_tensor = True)

bge_base_config = {
    "name":"bge_base_en_15",
    "long_name":"BAAI/bge-base-en-v1.5",
    "embeddings":None,
    "batch_embed_fun":bge_base_en_fun
}

method_configs = [openai_config, bge_large_config, bge_base_config, bge_small_config]


In [8]:
import torch
n = len(texts)

n_method = len(method_configs)
batch_size = 100

from tqdm import tqdm

corr_map = {}

for start_id in tqdm(range(0,n, batch_size)):
    end_id = min(start_id + batch_size, n)
    texts_batch = texts[start_id:end_id]

    method2embeddings = {}


    for method_config in method_configs:
        method_name = method_config["name"]
        if method_config["embeddings"] is None:
            embeddings = method_config["batch_embed_fun"](texts_batch)
        else:
            embeddings = torch.tensor(method_config["embeddings"][start_id:end_id]).cpu()

        if embeddings.device != torch.device("cpu"):
            embeddings = embeddings.cpu()

        method2embeddings[method_name] = embeddings

    for method_i in range(n_method):
        for method_j in range(method_i, n_method):
            corr_index = ( method_i, method_j )
            X_1 = method2embeddings[method_configs[method_i]["name"]]
            X_2 = method2embeddings[method_configs[method_j]["name"]]
            X1TX2 = X_1.T @ X_2
            if corr_index in corr_map:
                corr_map[corr_index] += X1TX2
            else:
                corr_map[corr_index] = X1TX2

for method_i in range(n_method - 1):
    for method_j in range(method_i + 1, n_method):
        corr_index = ( method_i, method_j )
        trans_index = (method_j, method_i )
        corr_map[trans_index] = corr_map[corr_index].T

100%|██████████| 1001/1001 [3:57:31<00:00, 14.24s/it]


In [ ]:
# print(method_config)

In [ ]:
# corr_map

In [9]:
import pickle

save_name_corr = "/content/drive/MyDrive/EmbedAdapter/corr_map_1218.pkl"

# Save corr_map to a pickle file
with open(save_name_corr, 'wb') as f:
    pickle.dump(corr_map, f)

In [10]:
import numpy as np

pseudo_inverses = {}

for i in range(n_method):
    # 计算每个方法自身相关性矩阵的伪逆
    corr_ii = corr_map[(i, i)].numpy()
    # print(corr_ii.dtype)
    pseudo_inv_ii = np.linalg.pinv(corr_ii)
    print(pseudo_inv_ii)
    # s_
    for j in range(n_method):
        if i != j:
            # 计算交叉相关性矩阵
            corr_ij = corr_map[(i, j)].numpy()
            # 计算伪逆
            pseudo_inverse_ij = pseudo_inv_ii @ corr_ij
            pseudo_key = (method_configs[i]["long_name"] , method_configs[j]["long_name"])
            pseudo_inverses[pseudo_key] = pseudo_inverse_ij

# 现在 pseudo_inverses 包含了所有相关性矩阵的伪逆


[[2.8719249 2.130673  2.0530279 ... 2.0937123 2.1580725 2.0796044]
 [2.1306732 2.8868496 2.127994  ... 2.1081119 2.1316993 2.1390023]
 [2.0530279 2.127994  2.748106  ... 2.0661223 2.1968846 2.0847766]
 ...
 [2.0937123 2.108112  2.0661223 ... 2.8677602 2.096815  2.0155046]
 [2.1580725 2.1316996 2.1968846 ... 2.096815  2.958371  2.136799 ]
 [2.0796044 2.1390023 2.0847766 ... 2.0155046 2.136799  2.7780182]]
[[1.1017386  0.6281255  0.6584521  ... 0.71208304 0.70639086 0.71830016]
 [0.6281255  1.0288153  0.60826343 ... 0.67744744 0.6306769  0.6903053 ]
 [0.65845215 0.60826343 1.0588397  ... 0.7084989  0.6915511  0.7444943 ]
 ...
 [0.71208304 0.6774474  0.7084989  ... 1.1594322  0.71852237 0.75818074]
 [0.70639086 0.6306769  0.6915511  ... 0.71852237 1.0598247  0.74178475]
 [0.71830016 0.6903053  0.7444943  ... 0.7581807  0.74178475 1.1932467 ]]
[[0.3300194  0.03984433 0.06276644 ... 0.02999802 0.06632057 0.02564175]
 [0.03984433 0.26601267 0.06214803 ... 0.0596054  0.03726843 0.0389826 ]
 [

In [ ]:
# corr_map_gpu={}
# save_name_inverse = "/content/drive/MyDrive/EmbedAdapter/corr_map_1218.pkl"
# import pickle
# with open('./corr_map.pkl', 'rb') as f:
#     corr_map_gpu=pickle.load(f)


In [ ]:

# for it in corr_map.keys():
#     print(torch.allclose(corr_map[it], corr_map_gpu[it], atol=1e-05, rtol=1e-03))


In [ ]:
# i,j=0,1
# corr_ii = corr_map[(0, 0)].cpu().numpy()
# pseudo_inv_ii = np.linalg.pinv(corr_ii)
# corr_ij = corr_map[(0, 1)].cpu().numpy()
#             # 计算伪逆
# pseudo_inverse_ij = pseudo_inv_ii @ corr_ij
# pseudo_key = (method_configs[0]["long_name"] , method_configs[1]["long_name"])
# print(pseudo_inverse_ij)

In [ ]:
# i,j=0,1
# corr_ii_gpu = corr_map_gpu[(0, 0)].cpu().numpy()
# pseudo_inv_ii_gpu = np.linalg.pinv(corr_ii_gpu)
# corr_ij_gpu = corr_map_gpu[(0, 1)].cpu().numpy()
#             # 计算伪逆
# pseudo_inverse_ij_gpu = pseudo_inv_ii_gpu @ corr_ij_gpu
# pseudo_key_gpu = (method_configs[0]["long_name"] , method_configs[1]["long_name"])
# print(pseudo_inverse_ij_gpu)

In [ ]:
# # print(torch.allclose(corr_ij,corr_ij_gpu, atol=1e-05, rtol=1e-03))
# print((corr_ii_gpu==corr_ii).all())
# print((pseudo_inv_ii_gpu==pseudo_inv_ii).all())
# print((( pseudo_inv_ii_gpu @ corr_ij_gpu)==( pseudo_inv_ii @ corr_ij)).any())


In [ ]:
# print(corr_ii_gpu.dtype,corr_ii.dtype)

In [ ]:
# for key in corr_map:
#     print(key, corr_map[key].shape)

In [11]:
import pickle

save_name_inverse = "/content/drive/MyDrive/EmbedAdapter/pseudo_inverses_1218.pkl"

# Save corr_map to a pickle file
with open(save_name_inverse, 'wb') as f:
    pickle.dump(pseudo_inverses, f)

In [12]:
for key in pseudo_inverses:
    print(key, pseudo_inverses[key].shape)

('openai', 'BAAI/bge-large-en-v1.5') (1536, 1024)
('openai', 'BAAI/bge-base-en-v1.5') (1536, 768)
('openai', 'BAAI/bge-small-en-v1.5') (1536, 384)
('BAAI/bge-large-en-v1.5', 'openai') (1024, 1536)
('BAAI/bge-large-en-v1.5', 'BAAI/bge-base-en-v1.5') (1024, 768)
('BAAI/bge-large-en-v1.5', 'BAAI/bge-small-en-v1.5') (1024, 384)
('BAAI/bge-base-en-v1.5', 'openai') (768, 1536)
('BAAI/bge-base-en-v1.5', 'BAAI/bge-large-en-v1.5') (768, 1024)
('BAAI/bge-base-en-v1.5', 'BAAI/bge-small-en-v1.5') (768, 384)
('BAAI/bge-small-en-v1.5', 'openai') (384, 1536)
('BAAI/bge-small-en-v1.5', 'BAAI/bge-large-en-v1.5') (384, 1024)
('BAAI/bge-small-en-v1.5', 'BAAI/bge-base-en-v1.5') (384, 768)


这里pseudo_inverses[(0,1)] 应该是乘在openai的embedding上可以转化为bge_small的

In [ ]:
!cp /content/pseudo_inverses.pkl /content/drive/MyDrive/unpublic